<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Graph Analysis using Script Table Operator(STO)
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Call Detail Records (CDRs) contain valuable information about communication patterns and interactions between users. By leveraging community detection algorithms on CDR data, businesses can gain insights into the underlying network structure and uncover meaningful communities within their user base.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The objective of this analysis is to identify distinct communities or groups of users within the CDR network. Communities are like smaller social circles or friend groups within the larger group of friends. It helps us understand how people naturally form different clusters based on their interactions and relationships. This analysis also identifies influential people in the graph.
<br>
<br>
By grouping users into communities based on their calling patterns, the business can better understand the dynamics and relationships among users, leading to several potential applications and benefits like Customer Segmentation, Fraud Detection, Network Optimization, Cross-Selling and Upselling Opportunities, Customer Support and Retention, etc.
</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo, we'll be using <b>Script Table Operator(STO)</b> to execute custom python scripts on Vantage. The STO operates by executing R and Python scripts from the command line of the Advanced SQL Engine underlying operating system, according to
the following sequence:
</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
  <li>The language script is installed on the Advanced SQL Engine of the target Teradata Vantage system via a call to
an External Stored Procedure (XSP)</li>
  <li>The script is invoked by executing a SQL query that calls the STO</li>
  <li>Each Advanced SQL Engine AMP provides its own portion of input table data, if any, to the script. The script
reads its input from the standard input STDIN</li>
  <li>Each Advanced SQL Engine AMP runs a different instance of the same script. Hence, the script execution is an
operation that scales through system architecture as the same script is run concurrently on all AMPs</li>
  <li>The script executes its code and sends its results to the standard output STDOUT. Each Advanced SQL Engine
AMP individually picks up the corresponding script instance results, and returns them to the STO</li>
</ol>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Downloading and installing additional software needed</b>

In [ ]:
%%capture
# '%%capture' suppresses the display of installation steps of the following packages
!pip install python-louvain
!pip install mplcursors

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import sys
import pandas as pd
import networkx as nx
from teradataml import *
import community
import matplotlib.pyplot as plt
import warnings
import mplcursors

# Suppress warnings
warnings.filterwarnings('ignore')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Graph_Analysis_PY_SQL.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_GraphAnalysis_cloud');"        # Takes 30 seconds
%run -i ../run_procedure.py "call get_data('DEMO_GraphAnalysis_local');"        # Takes 1 minute

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Using Script Table Operator</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Below is a sample of the data provided, where 'fromuserid' represents the caller and 'touserid' represents the callee.</p>

In [ ]:
DataFrame(in_schema('DEMO_GraphAnalysis', 'graph_data'))

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Community Detection using Louvain Algorithm</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The below cell will perform the following steps:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Set SEARCHUIFDBPATH to demo_user</li>
    <li>Install the communities.py file on Vantage</li>
    <li>If the file is already installed, it will remove the file and install it again. This ensures we always have latest script in Vantage.</li>
</ol>

Code: [communities.py](./communities.py)

In [ ]:
out=execute_sql("SET SESSION SEARCHUIFDBPATH = demo_user;")

try:
    execute_sql("CALL SYSUIF.install_file('communities', 'communities.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/communities.py');")
except:
    execute_sql("CALL SYSUIF.remove_file('communities', 1);")
    execute_sql("CALL SYSUIF.install_file('communities', 'communities.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/communities.py');")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The below cell will run the script installed in the above step and store the result in the community table.</p>

In [ ]:
query = '''CREATE TABLE community AS (
        SELECT * FROM SCRIPT(
        ON DEMO_GraphAnalysis.graph_data
        SCRIPT_COMMAND('/opt/teradata/languages/Python/bin/python3 ./demo_user/communities.py')
        RETURNS('node INT, community_id INT')
    ) AS tab
) with data;'''

try:
    execute_sql(query)
except:
    db_drop_table('community')
    execute_sql(query)

In [ ]:
DataFrame('community')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
We have a big group of customers, and they all like to talk to each other on the phone. When we talk about communities, we are interested in finding groups of users who are closely connected to each other and interact more frequently among themselves.
<br>
<br>
In our usecase, communities would be like different groups of users who are really close and talk to each other frequently. It's like having different circles of friends within the larger group. Some friends may belong to multiple communities if they have connections with people in different groups.</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Eigenvector Centrality</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Eigenvector Centrality is an algorithm that measures the transitive influence of nodes. Relationships originating from high-scoring nodes contribute more to the score of a node than connections from low-scoring nodes. A high eigenvector score means that a node is connected to many nodes who themselves have high scores.
    <br>
    <br>
The below cell will perform the following steps:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Set SEARCHUIFDBPATH to demo_user</li>
    <li>Install the centralities.py file on Vantage</li>
    <li>If the file is already installed, it will remove the file and install it again. This ensures we always have latest script in Vantage.</li>
</ol>

Code: [centralities.py](./centralities.py)

In [ ]:
out=execute_sql("SET SESSION SEARCHUIFDBPATH = demo_user;")

try:
    execute_sql("CALL SYSUIF.install_file('centralities', 'centralities.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/centralities.py');")
except:
    execute_sql("CALL SYSUIF.remove_file('centralities', 1);")
    execute_sql("CALL SYSUIF.install_file('centralities', 'centralities.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/centralities.py');")


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The below cell will run the script installed in the above step and store the result in the centrality table.</p>

In [ ]:
query = '''CREATE TABLE centrality AS (
        SELECT * FROM SCRIPT(
        ON DEMO_GraphAnalysis.graph_data
        SCRIPT_COMMAND('/opt/teradata/languages/Python/bin/python3 ./demo_user/centralities.py')
        RETURNS('node INT, centrality FLOAT')
    ) AS tab
) with data;'''

try:
    execute_sql(query)
except:
    db_drop_table('centrality')
    execute_sql(query)

In [ ]:
DataFrame('centrality').sort('centrality', ascending = False)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
We have a large group of customers, and they to talk to each other on the phone. Some of the customers are very popular and talk to lots of other customers, while others talk to only a few customers. Eigenvector centrality is a way to measure how important or popular each person is in this group based on the phone calls they make. So, in our graph with phone calls, eigenvector centrality helps us identify the people who are most connected to others and who have important connections. These people are considered more influential or popular in the group. This information can be used to efficiently target the the influential users and in turn the respective communities.</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Betweenness Centrality</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Betweenness centrality is a way of detecting the amount of influence a node has over the flow of information in a graph. It is often used to find nodes that serve as a bridge from one part of a graph to another. The algorithm calculates shortest paths between all pairs of nodes in a graph.
    <br>
    <br>
The below cell will perform the following steps:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Set SEARCHUIFDBPATH to demo_user</li>
    <li>Install the betweenness.py file on Vantage</li>
    <li>If the file is already installed, it will remove the file and install it again. This ensures we always have latest script in Vantage.</li>
</ol>

Code: [betweenness.py](./betweenness.py)

In [ ]:
out=execute_sql("SET SESSION SEARCHUIFDBPATH = demo_user;")

try:
    execute_sql("CALL SYSUIF.install_file('betweenness', 'betweenness.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/betweenness.py');")
except:
    execute_sql("CALL SYSUIF.remove_file('betweenness', 1);")
    execute_sql("CALL SYSUIF.install_file('betweenness', 'betweenness.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/betweenness.py');")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The below cell will run the script installed in the above step and store the result in the betweenness_table table.</p>

In [ ]:
query = '''CREATE TABLE betweenness_table AS (
        SELECT * FROM SCRIPT(
        ON DEMO_GraphAnalysis.graph_data
        SCRIPT_COMMAND('/opt/teradata/languages/Python/bin/python3 ./demo_user/betweenness.py')
        RETURNS('node INT, betweenness FLOAT')
    ) AS tab
) with data;'''

try:
    execute_sql(query)
except:
    db_drop_table('betweenness_table')
    execute_sql(query)

In [ ]:
DataFrame('betweenness_table').sort('betweenness', ascending = False)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
We have a group of customers, and they all like to talk to each other on the phone. Betweenness centrality is a way to measure how important or influential you are in this group based on the phone calls made by everyone. So, if you have a lot of customers who rely on you to connect with each other, it means you have high betweenness centrality. You're like a central hub in the group, helping users communicate and making sure everyone stays connected.
<br>
<br>
Betweenness centrality helps us identify the people who are important in keeping the group connected and maintaining communication. They are like the go-to person when someone needs to reach another user. They play a special role in making sure that everyone in the group stays connected and can talk to each other easily.</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Closeness Centrality</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Closeness centrality is a way of detecting nodes that are able to spread information very efficiently through a graph. The closeness centrality of a node measures its average farness (inverse distance) to all other nodes. Nodes with a high closeness score have the shortest distances to all other nodes.
    <br>
    <br>
The below cell will perform the following steps:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Set SEARCHUIFDBPATH to demo_user</li>
    <li>Install the closeness.py file on Vantage</li>
    <li>If the file is already installed, it will remove the file and install it again. This ensures we always have latest script in Vantage.</li>
</ol>

Code: [closeness.py](./closeness.py)

In [ ]:
out=execute_sql("SET SESSION SEARCHUIFDBPATH = demo_user;")

try:
    execute_sql("CALL SYSUIF.install_file('closeness', 'closeness.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/closeness.py');")
except:
    execute_sql("CALL SYSUIF.remove_file('closeness', 1);")
    execute_sql("CALL SYSUIF.install_file('closeness', 'closeness.py', 'cz!/home/jovyan/JupyterLabRoot/UseCases/Graph_Analysis/closeness.py');")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The below cell will run the script installed in the above step and store the result in the closeness_table table.</p>

In [ ]:
query = '''CREATE TABLE closeness_table AS (
        SELECT * FROM SCRIPT(
        ON DEMO_GraphAnalysis.graph_data
        SCRIPT_COMMAND('/opt/teradata/languages/Python/bin/python3 ./demo_user/closeness.py')
        RETURNS('node INT, closeness FLOAT')
    ) AS tab
) with data;'''

try:
    execute_sql(query)
except:
    db_drop_table('closeness_table')
    execute_sql(query)

In [ ]:
DataFrame('closeness_table').sort('closeness', ascending = False)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
We have a group of customers, and you all enjoy talking to each other on the phone. Closeness centrality is a way to measure how close or connected you are to all users in the group. When we talk about closeness centrality, we are interested in figuring out how quickly you can reach all the users when you make a phone call. If you can reach the users easily and quickly, then you have high closeness centrality.
<br>
<br>
Think of it like this: Imagine you want to invite all your friends to a party at your house. If your house is located in the middle of the neighborhood and all your friends live nearby, it will be easy and quick for them to come to your party. In the same way, if you can reach most of your friends with just a few phone calls, it means you are centrally located in the group and have high closeness centrality.
</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Visualization</b>

In [ ]:
%matplotlib widget

In [ ]:
G = nx.from_pandas_edgelist(DataFrame(in_schema('DEMO_GraphAnalysis', 'graph_data')).to_pandas().reset_index(), source='fromuserid', target='touserid', create_using=nx.Graph())
centrality = DataFrame('centrality').to_pandas().reset_index()
cdf = DataFrame('community').to_pandas().reset_index()
# cdf["node"] = cdf["node"].astype('int64')

# Define the leader nodes
df_sorted = pd.merge(DataFrame('community').to_pandas(), DataFrame('centrality').to_pandas(), left_on='node', right_on='node', how='inner', suffixes=('_community', '_centrality')).reset_index().sort_values('centrality', ascending=False)
leader_nodes = df_sorted.groupby('community_id')['node'].first().tolist()

# Draw the graph with nodes colored based on communities
pos = nx.spring_layout(G)
cmap = plt.get_cmap('tab10')  # Color map for communities

plt.figure(figsize=(10, 6))

# Create a dictionary to store community colors
community_colors = {}

init_nodes = nx.draw_networkx_nodes(G, pos, node_size=200)

for community_id in set(cdf.community_id):
    nodes = list(cdf[cdf['community_id'] == community_id].node)

    # Check if a node is a leader node
    node_sizes = [800 if node in leader_nodes else 200 for node in nodes]
    node_colors = [cmap(community_id) for node in nodes]

    scatter = nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=node_colors, node_size=node_sizes)

    # Store community color in the dictionary
    community_colors[community_id] = scatter.get_facecolor()[0]

nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5)
# nx.draw_networkx_labels(G, pos, font_size=8)

# Create a custom legend with community colors
legend_labels = [f'Community {community_id}' for community_id in set(cdf.community_id)]
custom_legend = [plt.Line2D([], [], marker='o', markersize=10, color=community_colors[community_id], linestyle='', label=label) for community_id, label in zip(set(cdf.community_id), legend_labels)]
plt.legend(handles=custom_legend)

plt.title('Graph with Communities')
plt.axis('off')

# Add hover functionality to nodes

def update_annot(sel):
    node_index = sel.target.index
    node_name = list(G.nodes)[node_index]
    text = 'Customer ID: ' + str(node_name) + ', EigenVector Centrality Score: ' + str(max(centrality[centrality['node'] == node_name]['centrality']))
    sel.annotation.set_text(text)

cursor = mplcursors.cursor(init_nodes, hover=True)
cursor.connect('add', update_annot)

plt.show()

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note</b>: Please hover over the nodes to see additional information.</i></p>
</div>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above graph displays the data in graph format. On hovering on the node, you might see Customer ID and the EigenVector Centrality Score i.e., the influence score. The larger nodes are influential and are connected to other influential nodes. These are the leader nodes of the respective communities.
    <br>
    <br>Targeting the leader of the communities in a telecom dataset can provide several benefits to a telecom company. Here are some ways it can help:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
  <li><strong>Influencing the community:</strong> Leaders of communities often hold significant influence over their members. By targeting and engaging with these leaders, a telecom company can leverage their influence to promote their products or services within the community. This can lead to increased brand awareness, customer acquisition, and loyalty.</li>
  
  <li><strong>Word-of-mouth marketing:</strong> Community leaders are typically respected and trusted individuals within their communities. When they endorse or recommend a telecom company's offerings, it can have a powerful impact on the community members' decisions. This word-of-mouth marketing can result in positive brand perception, organic referrals, and a higher likelihood of community members becoming customers.</li>
  
  <li><strong>Feedback and insights:</strong> Community leaders often have a deep understanding of their community's needs, preferences, and pain points. By establishing a relationship with these leaders, a telecom company can gain valuable insights and feedback about their products, services, and customer experiences. This information can be used to refine offerings, address specific community concerns, and tailor marketing strategies to better serve the target audience.</li>
  
  <li><strong>Co-creation and partnerships:</strong> Collaborating with community leaders can lead to mutually beneficial partnerships. Telecom companies can work with community leaders to co-create content, events, or initiatives that cater to the community's interests. This collaborative approach enhances engagement, fosters a sense of community, and establishes the telecom company as a trusted ally within the community.</li>
  
  <li><strong>Customer retention and satisfaction:</strong> Targeting community leaders allows a telecom company to prioritize customer satisfaction and address any issues or concerns promptly. By providing personalized support and assistance to these influential individuals, the company can improve overall customer experience, potentially leading to higher customer retention rates within the community.</li>
</ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In summary, targeting leaders of telecom communities enables a company to tap into their influence, leverage word-of-mouth marketing, gain valuable insights, foster partnerships, and enhance customer satisfaction. These efforts can result in increased brand visibility, customer acquisition, and long-term success for the telecom company.</p>


<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['community', 'centrality','betweenness_table','closeness_table']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_GraphAnalysis');"        # Takes 5 seconds

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>